#### Best Practices
1. Preprocessing and cleaning
2. Train Test Split
3. BOW,TFIDF , Word2vec
4. Train ML algorithms

In [1]:
# Loading dataset
import pandas as pd
data = pd.read_csv('all_kindle_review .csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [2]:
data=data[['reviewText', 'rating']]

In [3]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [4]:
data.shape

(12000, 2)

In [5]:
## Missing values
data.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [6]:
data['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [7]:
data['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [8]:
## Preprocessing And Cleaning

In [9]:
df = data

In [10]:
## Positive = 1 , negative = 0
df['rating'] = df['rating'].apply(lambda x: 0 if x < 3 else 1)


In [11]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [12]:
df['rating'].unique()

array([1, 0], dtype=int64)

In [13]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [14]:
## 1. Lower all the cases
df['reviewText'] = df['reviewText'].str.lower()

In [15]:
%pip install nltk
import nltk
from nltk.corpus import stopwords

Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [17]:
import re


In [18]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [19]:
from bs4 import BeautifulSoup
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
%pip install lxml


Note: you may need to restart the kernel to use updated packages.


In [21]:
## 2 . Clean the data , removing special characters
# removing special char


df['reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+', '', str(x)))
# remove stop words
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
# remove URLs
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'http\S+|www.\S+', '', x))
# remove Htmltags
df['reviewText']=df['reviewText'].apply(lambda x:BeautifulSoup(x,'lxml').get_text())
# removing aditional spaces
df['reviewText']=df['reviewText'].apply(lambda x:" ".join(x.split()))

In [22]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [23]:
## Lemmatization
from nltk.stem import WordNetLemmatizer

In [24]:
lemmatizer = WordNetLemmatizer()

In [25]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [26]:
df['reviewText'] = df['reviewText'].apply(lambda x: lemmatize_words(x))

In [27]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [28]:
## Train Test Split
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['rating'], test_size=0.2, random_state=42)

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [32]:
X_train_bow , X_test_bow

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64))

In [33]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_bow, y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf, y_train)

In [34]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [35]:
y_pred_bow = nb_model_bow.predict(X_test_bow)
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)

In [36]:
print('accuracy_score (BOW):', accuracy_score(y_test, y_pred_bow))
print('accuracy_score (TF-IDF):', accuracy_score(y_test, y_pred_tfidf))

accuracy_score (BOW): 0.575
accuracy_score (TF-IDF): 0.5791666666666667


In [37]:
import nltk

In [38]:
import gensim

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Tokenize the sentences
X_train_tokenized = [word_tokenize(sentence) for sentence in X_train]
X_test_tokenized = [word_tokenize(sentence) for sentence in X_test]


In [42]:
X_train_tokenized[:5], X_test_tokenized[:5]

([['looking',
   'forward',
   'book',
   'came',
   'double',
   'space',
   'every',
   'paragraph',
   'kindle',
   'edition',
   'since',
   'action',
   'move',
   'around',
   'formatting',
   'make',
   'story',
   'hard',
   'follow',
   'die',
   'hard',
   'like',
   'want',
   'botherits',
   'sad',
   'thing',
   'good',
   'book',
   'spoiled',
   'formatting',
   'fault',
   'author',
   'story',
   'good',
   'book',
   'energy',
   'read',
   'itive',
   'also',
   'emailed',
   'author'],
  ['already',
   'owned',
   'book',
   'spouse',
   'forgot',
   'already',
   'part',
   'library',
   'book',
   'unfortunate',
   'amazon',
   'safeguard',
   'place',
   'notify',
   'book',
   'already',
   'library'],
  ['cool',
   'forgot',
   'request',
   'rate',
   'came',
   'make',
   'mine',
   'unreliable',
   'rating',
   'doubt',
   'change',
   'rotation',
   'planet',
   'enough',
   'measure'],
  ['short',
   'short',
   'story',
   'basically',
   'scene',
   'par

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=5, min_count=1, workers=4)
# Transform sentences to vectors
def vectorize_sentences(sentences, model):
    vectors = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        vector = sum(model.wv[word] for word in words if word in model.wv) / len(words)
        vectors.append(vector)
    return vectors
X_train_w2v = vectorize_sentences(X_train, word2vec_model)
X_test_w2v = vectorize_sentences(X_test, word2vec_model)

In [40]:
X_train_w2v

[array([-0.3357794 ,  0.47981137,  0.15398547, -0.1930106 ,  0.28121477,
        -1.1198629 ,  0.60990065,  1.5374795 , -0.7710153 , -0.5067473 ,
        -0.2525904 , -0.9111414 , -0.28520006,  0.42391112,  0.37984496,
        -0.38474998,  0.14643194, -0.68742895, -0.25356227, -1.3787386 ,
         0.6246276 ,  0.3463614 ,  0.39546543, -0.4903878 , -0.04467716,
         0.20687547, -0.6175139 , -0.2413234 , -0.5788998 ,  0.23912849,
         0.72609216, -0.04293547,  0.21704051, -0.5334039 , -0.15353698,
         0.48831102,  0.28658384, -0.34621295, -0.28385347, -1.1294943 ,
         0.01322301, -0.49403208, -0.12030219,  0.15540342,  0.3893594 ,
        -0.17030686, -0.43538776, -0.06585187,  0.3968408 ,  0.64943963,
         0.25228536, -0.4268663 , -0.28277898, -0.3177441 , -0.18070251,
         0.32939833,  0.27529374, -0.17798996, -0.48540455,  0.45363045,
        -0.20470345,  0.19890083, -0.14156486,  0.00760142, -0.6476922 ,
         0.8791217 ,  0.27452013,  0.47451726, -0.9

In [43]:
from sklearn.naive_bayes import GaussianNB
nb_model_w2v = GaussianNB().fit(X_train_w2v, y_train)
y_pred_w2v = nb_model_w2v.predict(X_test_w2v)

In [44]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print('accuracy_score (Word2Vec):', accuracy_score(y_test, y_pred_w2v))
print('classification_report (Word2Vec):\n', classification_report(y_test, y_pred_w2v))
print('confusion_matrix (Word2Vec):\n', confusion_matrix(y_test, y_pred_w2v))

accuracy_score (Word2Vec): 0.6845833333333333
classification_report (Word2Vec):
               precision    recall  f1-score   support

           0       0.52      0.73      0.61       803
           1       0.83      0.66      0.74      1597

    accuracy                           0.68      2400
   macro avg       0.67      0.70      0.67      2400
weighted avg       0.73      0.68      0.69      2400

confusion_matrix (Word2Vec):
 [[ 585  218]
 [ 539 1058]]
